In [1]:
from __future__ import print_function
import time
import uiuc_incas_client
from uiuc_incas_client.rest import ApiException
from pprint import pprint

In [2]:
config = uiuc_incas_client.Configuration()
config.connection_pool_maxsize = 1
config.api_key['X-API-KEY'] = 'YOUR_API_KEY'
config.verify_ssl = False
client = uiuc_incas_client.ApiClient(config)
api_instance = uiuc_incas_client.MessageApi(client)

In [3]:
id_ = "message:twitter:0ab3ba38dfceeb5f3fe913900229a88ee3c45fb1"

In [4]:
try:
    # get message
    api_response = api_instance.message_id_get(id_, with_enrichment=True, dev=True) # dev indicates returning of all existing enrichments regardless of meta data existance.
    pprint(api_response)
except ApiException as e:
    print("Exception when calling MessageApi->message_id_get: %s\n" % e)

{'annotations': None,
 'author_actor': None,
 'content_text': 'RT @EmmanuelMacron Nous ne devons pas aujourd’hui céder à la '
                 "peur. C'est ce que nos assaillants attendent, et c'est leur "
                 'piège. https://t.co/LPIcMRc2sB',
 'data_tags': None,
 'embedded_urls': [],
 'enrichments': [{'enrichmentName': '11',
                  'enrichmentType': 'category',
                  'providerName': 'dmg',
                  'version': 'string'}],
 'extra_attributes': None,
 'geolocation': None,
 'id': '0ab3ba38dfceeb5f3fe913900229a88ee3c45fb1',
 'image_urls': [],
 'language': 'French',
 'media_type': 'Twitter',
 'media_type_attributes': {'reddit_data': None,
                           'twitter_data': {'engagement_parent_id': '855183122375299073',
                                            'engagement_type': 'retweet',
                                            'follower_count': 620,
                                            'following_count': 2730,
             

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:999: InsecureRequestWarning: Unverified HTTPS request is being made to host 'incas.cs.illinois.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [7]:
try:
    # get enrichments for this message
    api_response = api_instance.message_id_enrichments_get(id_, dev=True)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling MessageApi->message_id_enrichments_get: %s\n" % e)

[{'enrichmentName': '11',
  'enrichmentType': 'category',
  'providerName': 'dmg',
  'version': 'string'}]


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:999: InsecureRequestWarning: Unverified HTTPS request is being made to host 'incas.cs.illinois.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


## Add a new enrichment

In [8]:
enrichment_name = 'e1' # str
provider_name = 'p1' # str
version = '2.1' # str

In [9]:
try:
    # add the meta first
    meta = uiuc_incas_client.ArrayMessageEnrichmentMeta(enrichment_name=enrichment_name, enrichment_type='array', provider_name=provider_name, version=version)
    api_instance.message_enrichments_meta_post(meta)
except ApiException as e:
    print("Exception when calling MessageApi->message_enrichments_meta_post: %s\n" % e)

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:999: InsecureRequestWarning: Unverified HTTPS request is being made to host 'incas.cs.illinois.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [10]:
try:
    # insert a new enrichment
    body = uiuc_incas_client.ArrayMessageEnrichment(
        confidence=[0.5],
        enrichment_name=enrichment_name,
        enrichment_type='array',
        provider_name=provider_name,
        version=version,
        attribute_value=[1]
    )
    api_instance.message_id_enrichments_post(body, id_)
    
    # Show the content again
    api_response = api_instance.message_id_enrichments_get(id_, dev=True)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling MessageApi->message_id_enrichments_post: %s\n" % e)

[{'enrichmentName': '11',
  'enrichmentType': 'category',
  'providerName': 'dmg',
  'version': 'string'},
 {'attributeValue': [1.0],
  'confidence': [0.5],
  'enrichmentName': 'e1',
  'enrichmentType': 'array',
  'providerName': 'p1',
  'version': '2.1'}]


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:999: InsecureRequestWarning: Unverified HTTPS request is being made to host 'incas.cs.illinois.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:999: InsecureRequestWarning: Unverified HTTPS request is being made to host 'incas.cs.illinois.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [11]:
try:
    # update the enrichment
    body = uiuc_incas_client.ArrayMessageEnrichment(
        confidence=[0.5, 0.5],
        enrichment_name=enrichment_name,
        enrichment_type='array',
        provider_name=provider_name,
        version=version,
        attribute_value=[1, 2]  # change the value
    )
    api_instance.message_id_enrichments_put(body, id_)
    
    # Show the content again
    api_response = api_instance.message_id_enrichments_get(id_, dev=True)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling MessageApi->message_id_enrichments_put: %s\n" % e)

[{'enrichmentName': '11',
  'enrichmentType': 'category',
  'providerName': 'dmg',
  'version': 'string'},
 {'attributeValue': [1.0, 2.0],
  'confidence': [0.5, 0.5],
  'enrichmentName': 'e1',
  'enrichmentType': 'array',
  'providerName': 'p1',
  'version': '2.1'}]


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:999: InsecureRequestWarning: Unverified HTTPS request is being made to host 'incas.cs.illinois.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:999: InsecureRequestWarning: Unverified HTTPS request is being made to host 'incas.cs.illinois.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


## Delete an enrichment

In [13]:
try:
    # First delete the meta
    # api_instance.message_enrichments_meta_delete(enrichment_name, provider_name, version)
    
    # Here we use a batch delete, single delete is also available
    body = uiuc_incas_client.MessageEnrichmentsBatchDeleteBody(
        ids=[id_],
        enrichment_name=enrichment_name,
        provider_name=provider_name,
        version=version
    )
    api_instance.message_enrichments_batch_delete(body)
except ApiException as e:
    print("Exception when calling MessageApi->message_enrichments_batch_delete: %s\n" % e)

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:999: InsecureRequestWarning: Unverified HTTPS request is being made to host 'incas.cs.illinois.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:999: InsecureRequestWarning: Unverified HTTPS request is being made to host 'incas.cs.illinois.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [14]:
try:
    # get the current message number
    api_response = api_instance.message_count_get(media_type='Twitter')
    pprint(api_response)
except ApiException as e:
    print("Exception when calling MessageApi->message_count_get: %s\n" % e)

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:999: InsecureRequestWarning: Unverified HTTPS request is being made to host 'incas.cs.illinois.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


9


In [15]:
try:
    # get a batch of messages with ids
    body = uiuc_incas_client.MessageBatchGetBody(
        ids=["message:twitter:0ab3ba38dfceeb5f3fe913900229a88ee3c45fb1", "message:twitter:15ca66fd52c3fea8b3260120e79f09c87d88aec1"],
        dev=False,
        with_enrichment=True
    )
    api_response = api_instance.message_batch_get(body)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling MessageApi->message_batch_get: %s\n" % e)

[{'annotations': None,
 'author_actor': None,
 'content_text': 'RT @EmmanuelMacron Nous ne devons pas aujourd’hui céder à la '
                 "peur. C'est ce que nos assaillants attendent, et c'est leur "
                 'piège. https://t.co/LPIcMRc2sB',
 'data_tags': None,
 'embedded_urls': [],
 'enrichments': [],
 'extra_attributes': None,
 'geolocation': None,
 'id': '0ab3ba38dfceeb5f3fe913900229a88ee3c45fb1',
 'image_urls': [],
 'language': 'French',
 'media_type': 'Twitter',
 'media_type_attributes': {'reddit_data': None,
                           'twitter_data': {'engagement_parent_id': '855183122375299073',
                                            'engagement_type': 'retweet',
                                            'follower_count': 620,
                                            'following_count': 2730,
                                            'like_count': None,
                                            'mentioned_users': ['83812c529321337e2337b5642b973628'],

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:999: InsecureRequestWarning: Unverified HTTPS request is being made to host 'incas.cs.illinois.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
